In [9]:
%load_ext autoreload
%autoreload 2

%matplotlib nbagg

import time
import numpy as np
import matplotlib.pyplot as plt

from robmob.icp import icp
from robmob.robot import Robot
from robmob.sensors import HokuyoSensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
robot = Robot('localhost')
robot.connect()

In [11]:
hokuyo = HokuyoSensor()
robot.add_sensor(hokuyo)

In [12]:
def hokuyo_to_3d_homogeneous_matrix(hokuyo_data):
    ranges = np.array(hokuyo_data['ranges'])
    thetas = np.arange(hokuyo_data['angle_min'], hokuyo_data['angle_max']+hokuyo_data['angle_increment'], hokuyo_data['angle_increment'])
    xs = ranges * np.cos(thetas)
    ys = ranges * np.sin(thetas)
    
    pts = np.vstack((xs, ys, np.zeros(xs.shape), np.ones(xs.shape))).T
    
    # Remove the points that are at the origin.
    pts = pts[np.nonzero(np.logical_or(pts[:,0] != 0., pts[:,1] != 0.))]
    
    return pts

In [13]:
def plot_point_clouds(pcls):
    STYLES = ['r+', 'bx', 'g*']
    
    for i, pcl in enumerate(pcls):
        plt.plot(pcl[:,0], pcl[:,1], STYLES[i])
        plt.plot([0.0], [0.0], 'ko', markersize=10)
        
    plt.axis('equal')
    
    plt.show()

In [22]:
reading1 = hokuyo.peek_data()
pcl1 = hokuyo_to_3d_homogeneous_matrix(reading1)

In [23]:
reading2 = hokuyo.peek_data()
pcl2 = hokuyo_to_3d_homogeneous_matrix(reading2)

In [30]:
plot_point_clouds([pcl1, pcl2])

In [25]:
H, errs = icp(pcl1, pcl2)
print('La distance moyenne entre les points associés est de {}'.format(np.average(errs)))

Mean distance between associated points: 0.1400485654502107
Mean distance between associated points: 0.057090772602980684
Mean distance between associated points: 0.028797690730615864
Mean distance between associated points: 0.02117050783251823
Mean distance between associated points: 0.019527044044712415
Mean distance between associated points: 0.01920013817408489
La distance moyenne entre les points associés est de 0.01920013817408489


In [26]:
H

array([[ 0.99999859,  0.00167987,  0.        , -0.19545739],
       [-0.00167987,  0.99999859,  0.        , -0.00555033],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [27]:
moved_pcl = H.dot(pcl1.T)
moved_pcl = moved_pcl.T
moved_pcl

array([[ 2.90312106,  0.3157754 ,  0.        ,  1.        ],
       [ 2.84465677,  0.36359593,  0.        ,  1.        ],
       [ 2.79772514,  0.41117475,  0.        ,  1.        ],
       ...,
       [ 2.79068845, -0.59284994,  0.        ,  1.        ],
       [ 2.8241573 , -0.54472184,  0.        ,  1.        ],
       [ 2.87282023, -0.49816506,  0.        ,  1.        ]])

In [28]:
plot_point_clouds([pcl1, pcl2, moved_pcl])

In [29]:
pcl2

array([[ 2.92953642,  0.30877093,  0.        ,  1.        ],
       [ 2.84149636,  0.34987378,  0.        ,  1.        ],
       [ 2.79693654,  0.39419346,  0.        ,  1.        ],
       ...,
       [ 2.79814354, -0.59645657,  0.        ,  1.        ],
       [ 2.80410118, -0.54660489,  0.        ,  1.        ],
       [ 2.86777856, -0.5070906 ,  0.        ,  1.        ]])